In [16]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import os
import cv2
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [17]:
df = pd.read_csv(r"C:\Users\hp\Desktop\CaptchaDataset\CaptchaDataset\dataset1\mapping.csv")

In [18]:
symbols = string.ascii_lowercase + "0123456789" 
num_symbols = len(symbols)
img_shape = (50, 200, 1)

In [19]:
def create_model():
    img = layers.Input(shape=img_shape) 
    conv1 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  
    conv2 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)
    conv3 = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)  
    metrics = ["accuracy"] * 5
  
    flat = layers.Flatten()(mp3)
    outs = []
    for _ in range(5):
        dens1 = layers.Dense(256, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='softmax')(drop)

        outs.append(res)

    model = Model(img, outs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=metrics)
    return model

In [20]:
def preprocess_data(df, image_folder, num_symbols):
    n_samples = len(df)
    X = np.zeros((n_samples, 50, 200, 1))
    y = np.zeros((n_samples, num_symbols))

    for i, row in df.iterrows():
        # Read image as grayscale
        img = cv2.imread(os.path.join(image_folder, row['file_name']), cv2.IMREAD_GRAYSCALE)
        pic_target = row['captcha']
        if len(pic_target) < 6:
            # Scale and reshape image
            img = img / 255.0
            img = np.reshape(img, (50, 200, 1))
    
            targs = np.zeros((num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j] = ind
            X[i] = img
            y[i] = targs

    return X, y

In [21]:
image_folder = r'C:\Users\hp\Desktop\CaptchaDataset\CaptchaDataset\dataset1\images'  # Use raw string to preserve backslashes
X, y = preprocess_data(df, image_folder, num_symbols)
X_train, X_test, z_train, z_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
y_train = z_train.T
y_test = z_test.T

In [23]:
print(y_train)

[[12. 31. 30. ...  6. 15. 23.]
 [ 1. 33. 22. ... 33. 28. 28.]
 [15. 30. 33. ...  5. 12.  2.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [24]:
model = create_model()
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 50, 200, 1)        │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 50, 200, 32)       │             320 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 25, 100, 32)       │               0 │ conv2d_3[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 25, 100, 64)       │          18,496 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_4               │ (None, 13, 50, 64)        │               0 │ conv2d_4[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 13, 50, 128)       │          73,856 │ max_pooling2d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 13, 50, 128)       │             512 │ conv2d_5[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_5               │ (None, 7, 25, 128)        │               0 │ batch_normalization_1[0][… │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 22400)             │               0 │ max_pooling2d_5[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_10 (Dense)              │ (None, 256)               │       5,734,656 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_12 (Dense)              │ (None, 256)               │       5,734,656 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_14 (Dense)              │ (None, 256)               │       5,734,656 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_16 (Dense)              │ (None, 256)               │       5,734,656 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_18 (Dense)              │ (None, 256)               │       5,734,656 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_5 (Dropout)           │ (None, 256)               │               0 │ dense_10[0][0]             │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 28,812,724 (109.91 MB)

 Trainable params: 28,812,468 (109.91 MB)

 Non-trainable params: 256 (1.00 KB)

In [25]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [26]:
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], 
                 batch_size=32, epochs=25, verbose=1, 
                 validation_split=0.2, callbacks=[es])

Epoch 1/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 13s 332ms/step - dense_11_accuracy: 0.0529 - dense_13_accuracy: 0.0577 - dense_15_accuracy: 0.0499 - dense_17_accuracy: 0.0429 - dense_19_accuracy: 0.0563 - loss: 24.7381 - val_dense_11_accuracy: 0.0698 - val_dense_13_accuracy: 0.0872 - val_dense_15_accuracy: 0.0872 - val_dense_17_accuracy: 0.0523 - val_dense_19_accuracy: 0.0407 - val_loss: 17.6258
Epoch 2/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 291ms/step - dense_11_accuracy: 0.1013 - dense_13_accuracy: 0.1094 - dense_15_accuracy: 0.0943 - dense_17_accuracy: 0.1014 - dense_19_accuracy: 0.1154 - loss: 15.4640 - val_dense_11_accuracy: 0.1744 - val_dense_13_accuracy: 0.0407 - val_dense_15_accuracy: 0.0581 - val_dense_17_accuracy: 0.0930 - val_dense_19_accuracy: 0.0581 - val_loss: 17.4483
Epoch 3/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 311ms/step - dense_11_accuracy: 0.2595 - dense_13_accuracy: 0.2107 - dense_15_accuracy: 0.2060 - dense_17_accuracy: 0.1545 - dense_19_accuracy: 0.1596 - loss: 13.2287 - val_dense_11_

In [27]:
score = model.evaluate(X_test,[y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]], verbose=1)
print('Test loss and accuracy:', score)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - dense_11_accuracy: 0.9537 - dense_13_accuracy: 0.9267 - dense_15_accuracy: 0.8766 - dense_17_accuracy: 0.7920 - dense_19_accuracy: 0.8951 - loss: 1.9269
Test loss and accuracy: [1.8535107374191284, 0.9626168012619019, 0.9345794320106506, 0.8831775784492493, 0.8130841255187988, 0.8925233483314514]


In [28]:
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = img / 255.0
    else:
        print("Image not detected")
        return None
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, num_symbols))
    captcha = ""
    for i in range(5):
        captcha += symbols[np.argmax(ans[i])]
    return captcha

In [31]:
captcha_path = r'C:\Users\hp\Desktop\CaptchaDataset\CaptchaDataset\dataset1\images\1.png'
print(predict(captcha_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
p5g5m
